In [1]:
import pandas as pd
import numpy as np

In [2]:
meteorshowers=pd.read_csv("data/meteorshowers.csv")
cities=pd.read_csv("data/cities.csv")
moon_phases=pd.read_csv("data/moonphases.csv")
constellations=pd.read_csv("data/constellations.csv")

In [3]:
meteorshowers.info()
cities.info()
moon_phases.info()
constellations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   name                 5 non-null      object
 1   radiant              5 non-null      object
 2   bestmonth            5 non-null      object
 3   startmonth           5 non-null      object
 4   startday             5 non-null      int64 
 5   endmonth             5 non-null      object
 6   endday               5 non-null      int64 
 7   hemisphere           5 non-null      object
 8   preferredhemisphere  5 non-null      object
dtypes: int64(2), object(7)
memory usage: 492.0+ bytes
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256 entries, 0 to 255
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   city      256 non-null    object 
 1   latitude  256 non-null    float64
 2   country   256 non-null    object 
dtypes:

In [4]:
meteorshowers.head()

,name,radiant,bestmonth,startmonth,startday,endmonth,endday,hemisphere,preferredhemisphere
0,Lyrids,Lyra,april,april,21,april,22,northern,northern
1,Eta Aquarids,Aquarius,may,april,19,may,28,"northern, southern",southern
2,Orionids,Orion,october,october,2,november,7,"northern, southern","northern, southern"
3,Perseids,Perseus,august,july,14,august,24,northern,northern
4,Leonids,Leo,november,november,6,november,30,"northern, southern","northern, southern"


In [5]:
meteorshowers['startmonth'].unique()

array(['april', 'october', 'july', 'november'], dtype=object)

In [6]:
months = {'january':1,'february':2,'march':3,'april':4,'may':5,'june':6,'july':7,'august':8,'september':9,'october':10,'november':11,'december':12}

In [7]:
#딕셔너리를 이요한 mapping
meteorshowers['startmonth']=meteorshowers['startmonth'].map(months)
meteorshowers['bestmonth']=meteorshowers['bestmonth'].map(months)
meteorshowers['endmonth']=meteorshowers['endmonth'].map(months)

In [8]:
meteorshowers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   name                 5 non-null      object
 1   radiant              5 non-null      object
 2   bestmonth            5 non-null      int64 
 3   startmonth           5 non-null      int64 
 4   startday             5 non-null      int64 
 5   endmonth             5 non-null      int64 
 6   endday               5 non-null      int64 
 7   hemisphere           5 non-null      object
 8   preferredhemisphere  5 non-null      object
dtypes: int64(5), object(4)
memory usage: 492.0+ bytes


In [9]:
#날짜 형식의 데이터로 만들겠다.
meteorshowers['startdate']=2020*10000+meteorshowers['startmonth']*100+meteorshowers['startday']

In [10]:
meteorshowers['startdate']=pd.to_datetime(meteorshowers['startdate'], format='%Y%m%d')

In [11]:
meteorshowers['enddate']=pd.to_datetime(2020*10000+meteorshowers['endmonth']*100+meteorshowers['endday'],format='%Y%m%d')

In [12]:
moon_phases.info()
moon_phases['month']=moon_phases['month'].map(months)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 366 entries, 0 to 365
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   month         366 non-null    object
 1   day           366 non-null    int64 
 2   moonphase     50 non-null     object
 3   specialevent  10 non-null     object
dtypes: int64(1), object(3)
memory usage: 11.6+ KB


In [13]:
moon_phases['date']=pd.to_datetime(2020*10000+moon_phases['month']*100+moon_phases['day'],format='%Y%m%d')

In [14]:
moon_phases['moonphase'].unique()

array([nan, 'first quarter', 'full moon', 'third quarter', 'new moon'],
      dtype=object)

In [15]:
phases={'first quarter':0.5, 'full moon':1,'third quarter':0.5, 'new moon':0}

In [16]:
moon_phases['percentage']=moon_phases['moonphase'].map(phases)

In [17]:
moon_phases.drop(['month','day','moonphase','specialevent'],axis=1,inplace=True)

In [18]:
meteorshowers.drop(['startmonth','startday','endmonth','endday','hemisphere'],axis=1,inplace=True)

In [19]:
constellations.drop(['besttime'],axis=1,inplace=True)

In [20]:
for index,row in moon_phases.iterrows():
    print(index,row,sep='/n')
    break

0/ndate         2020-01-01
percentage          NaT
Name: 0, dtype: datetime64[ns]


In [21]:
for index,row in moon_phases.iterrows():
    print(index,row['percentage'])
    break

0 NaT


In [22]:
moon_phases1=moon_phases.copy()

In [23]:
lastphase=0
for index,row in moon_phases.iterrows():
    if pd.isnull(row['percentage']):
        moon_phases.loc[index,'percentage']=lastphase
    else:
        lastphase = moon_phases.loc[index,'percentage']

In [24]:
moon_phases.head()

,date,percentage
0,2020-01-01,0.0
1,2020-01-02,0.5
2,2020-01-03,0.5
3,2020-01-04,0.5
4,2020-01-05,0.5


* 위도를 체크하고 해당 도시에 표시되는 별자리를 확인 
* 그 후 별자리를 복사점으로 가지는 유성우 시작일과 종료일 정보확인
* 달의 위상 최저치 추출(달이 가장 어두운 날을 검색 및 출력)

## 도시의 위도 검색

In [25]:
cities.columns

Index(['city', 'latitude', 'country'], dtype='object')

In [26]:
cities[['city','latitude']]

,city,latitude
0,Abu Dhabi,24.47
1,Abuja,9.07
2,Accra,5.55
3,Adamstown,-25.07
4,Addis Ababa,9.02
...,...,...
251,Abidjan(former capital),6.82
252,Yaounde,3.87
253,Yaren(de facto),-0.53
254,Yerevan,40.18


In [27]:
c='Accra'
if c in cities.values:
    print(c)

Accra


In [28]:
cities['city'].unique()

array(['Abu Dhabi', 'Abuja', 'Accra', 'Adamstown', 'Addis Ababa',
       'Algiers', 'Alofi', 'Amman', 'Amsterdam', 'Andorra la Vella',
       'Ankara', 'Antananarivo', 'Apia', 'Ashgabat', 'Asmara', 'Asuncion',
       'Athens', 'Avarua', 'Baghdad', 'Baku', 'Bamako',
       'Bandar Seri Begawan', 'Bangkok', 'Bangui', 'Banjul', 'Basseterre',
       'Beijing', 'Beirut', 'Belgrade', 'Belmopan', 'Berlin', 'Bern',
       'Bishkek', 'Bissau', 'Bloemfontein(judicial)', 'Bogota',
       'Brasilia', 'Bratislava', 'Brazzaville', 'Bridgetown', 'Brussels',
       'Bucharest', 'Budapest', 'Buenos Aires', 'Cairo', 'Canberra',
       'Cape Town(legislative)', 'Caracas', 'Castries', 'Cayenne',
       'Charlotte Amalie', 'Chi-in-u', 'Conakry', 'Copenhagen', 'Dakar',
       'Damascus', 'Dhaka', 'Dili', 'Djibouti', 'Dodoma(official)',
       'Dar es Salaam(former capital; some government offices remaining)',
       'Doha', 'Douglas', 'Dublin', 'Dushanbe',
       'Edinburgh of the Seven Seas', 'El Aaiun(dec

In [29]:
#아래 2가지 방식이 결과가 같다.
#cities[cities['city']=='Abu Dhabia']['latitude']
cities.loc[cities['city']=='Abu Dhabi','latitude']

0    24.47
Name: latitude, dtype: float64

In [30]:
#위도 가져오는 사용자 정의 함수 정의
def predict_best_shower_viewing(city):

    shower_string = ""
    if city not in cities.values:
        shower_string = city + '는 현재 예측할 수 없습니다.'
        return shower_string

    
    latitude = cities.loc[cities['city']==city,'latitude'].iloc[0]
    constellations_list = constellations.loc[(constellations['latitudestart']>=latitude)&(constellations['latitudeend']<=latitude),'constellation'].to_list()

    if not constellations_list:
        shower_string = city + "에서는 볼 수 있는 유성우는 없습니다"
        return shower_string
    shower_string = city + '에서 유성우를 볼 수 있습니다.'
    
    return shower_string

incity = input('city>> ')    #function으로만들기
print(predict_best_shower_viewing(incity))

KeyboardInterrupt: Interrupted by user